# Fictional Pension Scheme Data Analysis

In this notebook, we work with some fictional pension scheme data to see what insights we can draw out.

We will work with: 
- a WIP log, which records details of work in progress and work completed to help the client team keep track of workflow and billing for individual cases
- data from a 2019 triennial valuation, which contains relevant data on all members of the scheme for the purposes of calculating the total scheme liability

### Import data

In [18]:
# Import pandas for data manipulation
import pandas as pd

# Read in data, parsing date columns as datetime objects to make calculations easier
wip = pd.read_csv("Data/fictional_wip.csv", parse_dates=["Date requested", "Deadline", "Date sent"])
val = pd.read_csv("Data/fictional_val_19.csv", parse_dates=["DOB", "DJC", "PSD", "DOL"])

In [19]:
# Inspect first few rows of WIP data
wip.head()

,Member name,Calculation type,Date requested,Deadline,Done by,Checked by,Reviewed by,Date sent,Size,Deadline met?
0,David Davidson,Transfer value,2019-05-01,2019-09-01,Andrew,Alice,Carol,2019-07-01,367120.0,Y
1,John Johnson,Retirement quote,2019-06-01,2019-08-01,Andrew,Alice,Carol,2019-08-01,15383.0,Y
2,Paul Paulson,Transfer value,2019-09-01,2019-11-01,Andrew,Alice,Carol,2019-01-13,309297.0,N
3,Mark Markson,Transfer value,2019-01-13,2019-01-17,Andrew,Alice,Carol,2019-01-16,27729.0,Y
4,James Jamesson,Retirement quote,2019-01-16,2019-01-18,Andrew,Alice,Bob,2019-01-19,5989.0,N


In [20]:
# Inspect first few rows of valuation data
val.head()

,First name,Surname,ID,Sex,DOB,Category,Insured?,DJC,PSD,DOL,Pension
0,Cathrin,Cathrinson,70000,F,1957-09-01,B,N,1986-02-01,1986-02-23,1989-02-22,2000.00
1,Cathryne,Cathryneson,70001,F,1966-04-07,A,N,1993-06-27,1994-03-19,2011-03-15,11333.33
2,Catrena,Catrenason,70002,F,1967-05-02,B,N,2007-01-26,2007-06-18,2016-06-15,6000.00
3,Cayan,Cayanson,70003,F,1961-05-19,A,N,2008-07-05,2008-10-15,2018-12-31,6810.96
4,Cecelia,Ceceliason,70004,F,1965-04-15,A,N,2007-05-04,2008-11-04,2014-10-04,4000.00


### Data dictionary

**WIP data:**
- Member name: name of member
- Calculation type: type of calculation requetsed by member or scheme administrator
- Deadline: date that the administrators require the calculation to be returned by
- Done by: staff member who performed the calculation
- Checked by: staff member who checked the calculation, once it had been done
- Reviewed by: staff member who reviewed and approved the calculation to be sent out, once it had been checked
- Date sent: date that the calculation was issued 
- Size: where applicable, the size of the transfer value (in £) or retirement quote (in £ per annum)
- Deadline met?: whether the calculation was sent out before or on the date of the given deadline

**Valuation data:**
- First name: member's first name
- Surname: member's surname
- ID: member's unique identification number within the scheme
- Sex: member's sex
- DOB: member's date of birth
- Category: member's benefit category.  Pension increases for category A are CPI-linked.  Pension increases for category B are RPI-linked.  Pension increases for category C are Big Mac Index-linked
- Insured?: whether the member is insured under a buy-in policy
- DJC: date upon which the member commenced company service
- PSD: date upon which the member commenced pensionable service
- DOL: date upon which the member left active service and became a deferred member of the scheme
- Pension: total pension in £ per annum that the member has accrued as at their date of leaving

### Exploratory Data Analysis
#### Missing values

In [21]:
# Check for missing values
wip.isna().sum()

Member name          0
Calculation type     0
Date requested       0
Deadline             0
Done by              0
Checked by           0
Reviewed by          1
Date sent            0
Size                15
Deadline met?        0
dtype: int64

There appear to be 16 missing values, 15 of which are in the Size column and one of which is in the Reviewed By column.  Inspect further:

In [22]:
wip[wip["Size"].isna()]

,Member name,Calculation type,Date requested,Deadline,Done by,Checked by,Reviewed by,Date sent,Size,Deadline met?
7,Steven Stevenson,Pension sharing order,2019-01-27,2019-01-31,Andrew,Bob,Carol,2019-01-30,NaN,Y
9,Stephen Stephenson,Augmentation,2019-09-02,2019-11-02,Andrew,Alice,Bob,2019-02-13,NaN,N
11,Craig Craigson,Augmentation,2019-02-14,2019-02-17,Alice,Andrew,Carol,2019-02-18,NaN,N
14,Christopher Christopherson,Augmentation,2019-02-26,2019-02-28,Andrew,Alice,Bob,2019-02-03,NaN,N
16,Colin Colinson,LRF calculation,2019-11-03,2019-03-14,Andrew,Alice,Bob,2019-03-13,NaN,Y
24,Neil Neilson,Pension sharing order,2019-04-05,2019-09-05,Andrew,Alice,Bob,2019-08-05,NaN,Y
25,Barry Barryson,Augmentation,2019-05-05,2019-10-05,Andrew,Bob,Alice,2019-07-05,NaN,Y
36,Kenneth Kennethson,Augmentation,2019-11-06,2019-06-16,Andrew,Alice,Carol,2019-06-13,NaN,Y
44,Ross Rossson,Augmentation,2019-06-08,2019-10-08,Dave,Andrew,Alice,2019-09-08,NaN,Y
45,Jonathan Jonathanson,LRF calculation,2019-07-08,2019-09-08,Dave,Andrew,Bob,2019-09-08,NaN,Y


It seems like the Size for these calcaultions is either not applicable or has not been recorded.    

How has the number and type of case Andrew has done/checked change over time?  Show increased responsibility, new employee, can we show it on one graph? number of cases each person is involved with over each month?


How does the percentage of deadlines hit vary over time?  can we identify any time peroods where the clent team was struggling?  If so, we could look into what was going wrong and try to address the problem

Do certain cases take longer/do we fail to hit the deadline on certain cases more often than usual?  Perhaps we need additionsl training on these

Are there any trends in the size column?  Which sizes are most common for TVs/retirement quotes?



Go back into the data and insert trends - bit cheeky as we will know what we're looking for but can't use real client data and trends are based off of real world experience! 